In [1]:
import requests
import html
import urllib.request
import urllib.error
import urllib.parse
from bs4 import BeautifulSoup as bs

import time
import os
import seaborn as sns
import pandas as pd
import scipy as sc
import numpy as np

import statsmodels.formula.api as sm

import matplotlib.pyplot as plt 
plt.style.use('ggplot')
%matplotlib inline  
plt.rcParams['figure.figsize'] = (10, 6) 

In [2]:
#creating the list of Billboard urls
billboardUrls = []
years = ["2016","2017","2018"]
for i in years:
    url = 'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_' + str(i)
    billboardUrls.append(url)   
print(billboardUrls)  


['https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2016', 'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2017', 'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_2018']


In [3]:
#saving 3 billboard pages on the laptop   
def save_html(url):
    response1 = urllib.request.urlopen(url)
    webContent = response1.read()
    
    f = open(url[url.index('Billboard'):]+'.html', 'wb') #if there is no such file the function open creates it 
    f.write(webContent)
    f.close

for url in billboardUrls:
    save_html(url)

In [4]:
#where the data (html files) is stored
DATA_PATH = "/Users/alisamakarova"
#Creating an array for saving the html code of the pages
source = []
#Loading each of the 10 pages with a for loop
for x in range(6, 9):
    url2 = DATA_PATH + "/Billboard_Year-End_Hot_100_singles_of_201" + str(x) + ".html"
    HtmlFile = open(url2, 'r', encoding='utf-8')
    source.append(bs(HtmlFile, "html.parser"))
print(source)      

[<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Billboard Year-End Hot 100 singles of 2016 - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XnEBSwpAML8ABBPj45AAAAAB","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Billboard_Year-End_Hot_100_singles_of_2016","wgTitle":"Billboard Year-End Hot 100 singles of 2016","wgCurRevisionId":927410725,"wgRevisionId":927410725,"wgArticleId":52434959,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategori

In [5]:
#creating lists of lists for each year
songs={"2016": [],"2017": [], "2018":[]}
artists={"2016": [],"2017": [], "2018":[]}

for i in range(6,9):
    content = open("{}/Billboard_Year-End_Hot_100_singles_of_201{}.html".format(DATA_PATH, i), encoding='utf-8')
    soup_html = bs(content, "html.parser")

    titles=soup_html.findAll('td')
    for title in titles[0:len(titles)-5:2]:
        songs["201{}".format(i)].append(title.text.replace('\n',''))
    for title in titles[1:len(titles)-5:2]:    
        artists["201{}".format(i)].append(title.text.replace('\n',''))
 

print(artists["2018"][0])        
        

Drake


In [6]:
#creating a list of a df for each year
songs_df={"2016_df": pd.DataFrame(),"2017_df": pd.DataFrame(), "2018_df":pd.DataFrame()}
for key, value in songs_df.items():
    print(key)
    songs_df[key]['Title'] = songs["2016"]
    songs_df[key]['Artist'] = artists["2016"]
songs_df["2016_df"].head()

2016_df
2017_df
2018_df


,Title,Artist
0,"""Love Yourself""",Justin Bieber
1,"""Sorry""",Justin Bieber
2,"""One Dance""",Drake featuring Wizkid and Kyla
3,"""Work""",Rihanna featuring Drake
4,"""Stressed Out""",Twenty One Pilots


In [8]:
songs_urls={"2016": [],"2017": [], "2018": []}

for i in range(6,9):
    content = open("{}/Billboard_Year-End_Hot_100_singles_of_201{}.html".format(DATA_PATH, i), encoding='utf-8')
    soup_html = bs(content, "html.parser")

    titles=soup_html.findAll('td')
    for title in titles[0:len(titles)-5:2]:
        songs_urls["201{}".format(i)].append('https://en.wikipedia.org'+str(title)[str(title).index('href')+6:str(title).index('title')-2])
        
songs_urls        

{'2016': ['https://en.wikipedia.org/wiki/Love_Yourself',
  'https://en.wikipedia.org/wiki/Sorry_(Justin_Bieber_song)',
  'https://en.wikipedia.org/wiki/One_Dance',
  'https://en.wikipedia.org/wiki/Work_(Rihanna_song)',
  'https://en.wikipedia.org/wiki/Stressed_Out',
  'https://en.wikipedia.org/wiki/Panda_(song)',
  'https://en.wikipedia.org/wiki/Hello_(Adele_song)',
  'https://en.wikipedia.org/wiki/Don%27t_Let_Me_Down_(The_Chainsmokers_song)',
  'https://en.wikipedia.org/wiki/Can%27t_Stop_the_Feeling!',
  'https://en.wikipedia.org/wiki/Closer_(The_Chainsmokers_song)',
  'https://en.wikipedia.org/wiki/Cheap_Thrills_(song)',
  'https://en.wikipedia.org/wiki/7_Years_(Lukas_Graham_song)',
  'https://en.wikipedia.org/wiki/Needed_Me',
  'https://en.wikipedia.org/wiki/My_House_(Flo_Rida_song)',
  'https://en.wikipedia.org/wiki/I_Took_a_Pill_in_Ibiza',
  'https://en.wikipedia.org/wiki/Work_from_Home',
  'https://en.wikipedia.org/wiki/This_Is_What_You_Came_For',
  'https://en.wikipedia.org/wiki

In [11]:
for k in range (6,9):
    for i in range(0, 100):
        url=songs_urls["201{}".format(k)][i]
        temp=str(url).split('/')
        name_of_song=temp[len(temp)-1]
        f = open("201"+str(k)+"_"+name_of_song+'.html', 'wb')
        page = urllib.request.urlopen(url)
        pagetext = page.read()
        f.write(pagetext)
        f.close()

AttributeError: '_io.BufferedWriter' object has no attribute 'save'